In [4]:
import pandas as pd
import numpy as np
import json

from vncorenlp import VnCoreNLP
vnsegmentor = VnCoreNLP("../../../VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg")

In [4]:
train_data = []
with open("./data/public_test/public_test.jsonl", "r") as f:
    for line in f:
        train_data.append(json.loads(line))

In [7]:
class SquadDataset:
    def __init__(self):
        self.squad_dict = {
            "version": None,
            "data": [],
        }
        
    @staticmethod
    def create_data():
        return {
            "title": None,
            "paragraphs": [],
        }
    
    @staticmethod
    def create_paragraph():
        return {
            "context": "",
            "qas": [],
        }
    
    @staticmethod
    def create_question(question):
        return {
            "question": question,
            "id": None,
            "is_impossible": None,
            "answers": [],
        }
    
    @staticmethod
    def create_answer():
        return {
            "answer_start": None,
            "text": None,
        }

In [6]:
def add_question(qas_list, question):
    global qid
    question["id"] = qid
    qid += 1
    qas_list.append(question)

In [16]:
from bs4 import BeautifulSoup
import re

squad_dataset = SquadDataset()
len_context = []
qid = 0

for data_id, data in enumerate(train_data):
    squad_data = SquadDataset.create_data()
    squad_data["title"] = data["original_doc"]["_source"]["description"]
    squad_paragraph = SquadDataset.create_paragraph()

    event_texts = {0: ""}
    event_paddings = {0: 0}
    event_ids = []
    e_id = None
    
    for html in data["html_annotation"]:
        for e in BeautifulSoup(html):
            try:
                e_id = int(e["event_id"])
                event_ids.append(e_id)
                event_texts[e_id] = ""
            except: pass
            if e_id is None:
                continue
            content = vnsegmentor.tokenize(e.text.strip() + " ")
            content += " ".join(" ".join(x) for x in content)
            event_texts[e_id] += content

    event_ids.sort()
    for i, event in enumerate(event_ids):
        cur_event_ids = event_ids[i]
        pre_event_ids = event_ids[i - 1] if i != 0 else 0
        event_paddings[cur_event_ids] = event_paddings[pre_event_ids] + len(event_texts[pre_event_ids])
        squad_paragraph["context"] += event_texts[cur_event_ids]

    summary = data["match_summary"]
    
    # add question answer pairs
    team1 = SquadDataset.create_question("team1")
    team1 = parse_answers(summary["players"]["team1"], summary["players"]["ref_event_ids"], event_texts, event_paddings)
    team2 = SquadDataset.create_question("team2")
    team2 = parse_answers(summary["players"]["team2"], summary["players"]["ref_event_ids"], event_texts, event_paddings)
    
    score1 = SquadDataset.create_question("score1")
    score1 = parse_answers(summary["score_board"]["score1"], summary["score_board"]["ref_event_ids"], event_texts, event_paddings)
    socre2 = SquadDataset.create_question("score2")
    score2 = parse_answers(summary["score_board"]["score2"], summary["score_board"]["ref_event_ids"], event_texts, event_paddings)
    
    for score in summary["score_list"]:
        if score["team"] == summary["players"]["team1"]:
            score_players = SquadDataset.create_question("score player team1")
        else:
            score_players = SquadDataset.create_question("score player team2")
        
    
    squad_data["paragraphs"] = [squad_paragraph]

    if squad_data_valid:
        len_context.append(len(squad_paragraph["context"].split()))
        squad_dataset.squad_dict["data"].append(squad_data)

In [17]:
def train_dev_split(data_list, dev_size):
    num_data = len(data_list)
    num_train = num_data - int(num_data * dev_size)
    ids = np.random.permutation(range(num_data))
    train_list = [data_list[x] for x in ids[:num_train]]
    dev_list = [data_list[x] for x in ids[num_train:]]
    print("Split:", len(train_list), len(dev_list))
    return train_list, dev_list

train_data, dev_data = train_dev_split(squad_dataset.squad_dict["data"], dev_size=0.2)
train_dict = {"version": 2.0, "data": train_data}
dev_dict = {"version": 2.0, "data": dev_data}

Split: 520 129


In [18]:
with open("squad_train.json", "w") as f:
    json.dump(train_dict, f, indent=2, ensure_ascii=False)

with open("squad_dev.json", "w") as f:
    json.dump(dev_dict, f, indent=2, ensure_ascii=False)

In [19]:
pd.Series(len_context).describe()

count     649.000000
mean      550.237288
std       232.659667
min        63.000000
25%       385.000000
50%       528.000000
75%       692.000000
max      1776.000000
dtype: float64